# Left Ventricle Full Quantification (LVQuan18) Challenge MICCAI 2018

Accurate cardiac left ventricle (LV) quantification is among the most clinically important and most frequently demanded tasks for identification and diagnosis of cardiac diseases and is of great interest in the research community of medical image analysis.

### The task

We must segment the LV from the SAX MR sequences of the heart. The focus will be on full quantification of LV, i.e., all clinical significant LV indices regarding to the anatomical structure of LV are investigated in addition to the frequently studied LV volume;

### The dataset

A training dataset with processed SAX MR sequences of 145 subjects from clinical environment is used for model learning and validation. For each subject, 20 frames are included for the whole cardiac cycle. All ground truth values of the above-mentioned LV indices are provided for every single frame. More details about the training dataset can be found in the document of the DIG-Cardiac dataset. 